In [ ]:
# Installing libraries
!pip install pathway bokeh

In [ ]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [ ]:
# Loading data
data = pd.read_csv('/content/dataset.csv')
data

In [ ]:
# Data info
data.info()

In [ ]:
# Parsing and sorting according to timestamp
data['Timestamp'] = pd.to_datetime(data['LastUpdatedDate'] + ' ' + data['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')
data = data.sort_values('Timestamp').reset_index(drop=True)
data.head()

In [ ]:
# Filtering data for demand based price function model
data[["SystemCodeNumber", "Timestamp", "Occupancy", "Capacity", "QueueLength", "TrafficConditionNearby", "IsSpecialDay", "VehicleType"]].to_csv("demand_price_parking_stream.csv", index=False)

In [ ]:
# Defining schema for demand based price model
class Demand_Price_ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: str
    IsSpecialDay: int
    VehicleType: str

In [ ]:
# Simulating real time data stream using 100 rows
demand_price_data_parking = pw.demo.replay_csv("demand_price_parking_stream.csv", schema = Demand_Price_ParkingSchema, input_rate=100)

In [ ]:
# Formatting date and time
fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = demand_price_data_parking.with_columns(
    t = demand_price_data_parking.Timestamp.dt.strptime(fmt),
    day = demand_price_data_parking.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00"))

In [ ]:
# Calculating utilization ratio
data_utilization = data_with_time.with_columns(
    utilization=pw.this.Occupancy / pw.this.Capacity
)

In [ ]:
# Demand Based Price Function
@pw.udf
def demand_price(system_code, t, occupancy, capacity, queue, traffic, is_special, vehicle_type) -> float:
    vehicle_weights = {"car": 1.0, "bike": 0.6, "truck": 1.5}
    traffic_encoding = {"low": 0.2, "medium": 0.5, "high": 1.0}

    alpha = 1.5
    beta = 0.5
    gamma = 1.0
    delta = 2.0
    epsilon = 1.0
    base_price = 10.0
    lambda_ = 0.8

    occ_ratio = occupancy / capacity
    traffic_val = traffic_encoding.get(str(traffic).lower(), 0.5)
    vtype_weight = vehicle_weights.get(str(vehicle_type).lower(), 1.0)

    demand = (
        alpha * occ_ratio +
        beta * queue -
        gamma * traffic_val +
        delta * int(is_special) +
        epsilon * vtype_weight
    )

# Normalizing Demand Using Sigmoid
    norm_demand = 1 / (1 + np.exp(-demand))
    price = base_price * (1 + lambda_ * norm_demand)

# Price Range Bounding
    return max(min(price, 2 * base_price), 0.5 * base_price)

price_stream_demand = data_with_time.with_columns(
    price = demand_price(
        pw.this.SystemCodeNumber,
        pw.this.t,
        pw.this.Occupancy,
        pw.this.Capacity,
        pw.this.QueueLength,
        pw.this.TrafficConditionNearby,
        pw.this.IsSpecialDay,
        pw.this.VehicleType
    )
)

pw.run()

In [ ]:
# Grouping by lot and determining mean price for each day
price_stream_with_day = price_stream_demand.with_columns(
    date = pw.this.t.dt.strftime("%Y-%m-%dT00:00:00")
)

daily_price_stream = price_stream_with_day.groupby(
    pw.this.SystemCodeNumber, pw.this.date
).reduce(
    SystemCodeNumber = pw.this.SystemCodeNumber,
    t = pw.reducers.max(pw.this.t),
    sum_price = pw.reducers.sum(pw.this.price),
    count_price = pw.reducers.count()
).with_columns(
    price = pw.this.sum_price / pw.this.count_price
)

In [ ]:
# Plotting Demand Based Price Function (Daily)
from bokeh.models import ColumnDataSource
pn.extension()

parking_lots = ['BHMBCCMKT01', 'BHMNCPHST01', 'BHMMBMMBX01', 'BHMNCPNST01',
       'Shopping', 'BHMEURBRD01', 'Broad Street', 'Others-CCCPS8',
       'Others-CCCPS105a', 'Others-CCCPS119a', 'BHMBCCTHL01',
       'Others-CCCPS135a', 'Others-CCCPS202', 'Others-CCCPS98']

def lot_price_plot(system_code):
    def plot_func(source):
        fig = bokeh.plotting.figure(
            height=300,
            width=500,
            title=f"Price Dynamics: {system_code}",
            x_axis_type="datetime",
            y_range = (10, 20)
        )
        fig.line("t", "price", source=source, line_width=2, color="navy")
        fig.scatter("t", "price", source=source, size=5, color="red")
        return fig
    return plot_func

viz_dict = {
    lot: daily_price_stream.filter(pw.this.SystemCodeNumber == lot).plot(
        lot_price_plot(lot), sorting_col="t"
    )
    for lot in parking_lots
}

column1 = [viz_dict[lot] for lot in parking_lots[:7]]
column2 = [viz_dict[lot] for lot in parking_lots[7:]]

dashboard = pn.Row(
    pn.Column(*column1),
    pn.Column(*column2)
)

dashboard.servable()

In [ ]:
# Run Demand Based Price Function
%%capture --no-display
pw.run()